In [10]:
from langchain_google_genai import GoogleGenerativeAI , GoogleGenerativeAIEmbeddings
from getpass import getpass
from os import environ

In [8]:
api_key = getpass("Google api key")

In [12]:
environ["GOOGLE_API_KEY"] = api_key

In [4]:
model = llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
model.invoke("hey hi")

'Hello! How can I assist you today?'

In [13]:
import pandas as pd

In [19]:
df = pd.read_csv("dataset.csv")
df.head(10)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("dataset.csv",)
data = loader.load()[:600]

In [103]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain.vectorstores import FAISS
db = FAISS.from_documents(data,embedding=embeddings)
db.save_local("local_db")

In [104]:
db = FAISS.load_local("local_db")
first_data = db.similarity_search("my head is shaking")[0]
first_data.page_content
first_data.metadata


TypeError: FAISS.load_local() missing 1 required positional argument: 'embeddings'

In [58]:
retreiver = db.as_retriever()
retreiver.invoke("skin,shivering")

[Document(page_content='Disease: Allergy\nSymptom_1: shivering\nSymptom_2: chills\nSymptom_3: watering_from_eyes\nSymptom_4: \nSymptom_5: \nSymptom_6: \nSymptom_7: \nSymptom_8: \nSymptom_9: \nSymptom_10: \nSymptom_11: \nSymptom_12: \nSymptom_13: \nSymptom_14: \nSymptom_15: \nSymptom_16: \nSymptom_17: ', metadata={'source': 'dataset.csv', 'row': 11}),
 Document(page_content='Disease: Allergy\nSymptom_1: shivering\nSymptom_2: chills\nSymptom_3: watering_from_eyes\nSymptom_4: \nSymptom_5: \nSymptom_6: \nSymptom_7: \nSymptom_8: \nSymptom_9: \nSymptom_10: \nSymptom_11: \nSymptom_12: \nSymptom_13: \nSymptom_14: \nSymptom_15: \nSymptom_16: \nSymptom_17: ', metadata={'source': 'dataset.csv', 'row': 15}),
 Document(page_content='Disease: Allergy\nSymptom_1: continuous_sneezing\nSymptom_2: shivering\nSymptom_3: chills\nSymptom_4: \nSymptom_5: \nSymptom_6: \nSymptom_7: \nSymptom_8: \nSymptom_9: \nSymptom_10: \nSymptom_11: \nSymptom_12: \nSymptom_13: \nSymptom_14: \nSymptom_15: \nSymptom_16: \nSym

In [77]:
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [99]:
prompt_template = """If the context is not relevant, 
        please answer the question by using your own knowledge about the topic
        output format 
        ```json
                disease:"",
                other symptom:[],
                doctor type:Suggest this based on your knowledge,
                remedies:Suggest this based on your knowledge,
        ```
        {context}
        
        Question: {question}
        """
prompt = ChatPromptTemplate.from_template(prompt_template)

In [100]:
chain = (
    {"context": retreiver, "question": RunnablePassthrough()}
    | prompt
    | model
    | JsonOutputParser()
)

In [101]:
chain.invoke("Itching")

{'disease': 'Fungal infection',
 'other symptom': ['skin_rash', 'dischromic _patches', 'nodal_skin_eruptions'],
 'doctor type': 'Dermatologist',
 'remedies': ['Apply an antifungal cream or ointment to the affected area.',
  'Take oral antifungal medication if the infection is severe.',
  'Keep the affected area clean and dry.']}